In [1]:
import numpy as np
import cv2
import sklearn.metrics as metrics
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

LABEL_PATH = "./LaneDetection/seg_data/test/Label/"
OUTPUT_PATH = "./LaneDetection/seg_data/test/outputNew/"
NO_OF_CLASSES = 2

LABEL_SUFFIX = '.jpg'
OUTPUT_SUFFIX = '.png'


confusion_matrix = np.zeros([NO_OF_CLASSES, NO_OF_CLASSES])

output_file_name_list = Path(OUTPUT_PATH).glob('*' + OUTPUT_SUFFIX)
print(output_file_name_list)
num_images_skipped = 0
for output_file_name in output_file_name_list:
    print(output_file_name)
    
    ground_truth_name = str(Path(LABEL_PATH) / (str(output_file_name.name).split(OUTPUT_SUFFIX)[0] + LABEL_SUFFIX))
    output_image = cv2.imread(str(output_file_name))[:, :, 0]
    ground_truth = cv2.imread(ground_truth_name)[:, :, 0]
    
    
    if output_image is None:
        
        print('No output image, skipping : ' + output_file_name)
        num_images_skipped = num_images_skipped + 1
        continue
    
    if ground_truth is None:
        
        print('No ground truth image, skipping : ' + ground_truth_name)
        num_images_skipped = num_images_skipped + 1
        continue

    ground_truth = (ground_truth > 200) == True
    output_image = output_image == 255

    confusion_matrix += metrics.confusion_matrix(ground_truth.reshape(-1), output_image.reshape(-1), range(NO_OF_CLASSES))
    print(confusion_matrix)


total_predictions = np.sum(confusion_matrix)
mean_accuracy = mean_iou = mean_dice = 0
for class_id in range(0, NO_OF_CLASSES):

    tp = confusion_matrix[class_id, class_id]
    fp = np.sum(confusion_matrix[: class_id, class_id]) + np.sum(confusion_matrix[class_id + 1 :, class_id])
    fn = np.sum(confusion_matrix[class_id, : class_id]) + np.sum(confusion_matrix[class_id, class_id + 1 :])
    tn = total_predictions - tp - fp - fn
    
    accuracy = (tp + tn) / (tn + fn + tp + fp) 
    mean_accuracy += accuracy

    if ((tp + fp + fn) != 0):
        iou = (tp) / (tp + fp + fn)
        dice = (2 * tp) / (2 * tp + fp + fn)
    else:
        # When there are no positive samples and model is not having any false positive, we can not judge IOU or Dice score
        # In this senario we assume worst case IOU or Dice score. This also avoids 0/0 condition
        iou = 0.0
        dice = 0.0

    mean_iou += iou
    mean_dice += dice

    print("CLASS: {}: Accuracy: {}, IOU: {}, Dice: {}".format(class_id, accuracy, iou, dice))

mean_accuracy = mean_accuracy / (NO_OF_CLASSES)
mean_iou = mean_iou / (NO_OF_CLASSES)
mean_dice = mean_dice / (NO_OF_CLASSES)
print("Mean Accuracy: {}, Mean IOU: {}, Mean Dice: {}".format(mean_accuracy, mean_iou, mean_dice))

<generator object Path.glob at 0x0000022E8E67BDC8>
F:\Here\LaneDetection\seg_data\train\01759047-d88724fe.png
[[908262.   1209.]
 [  5760.   6369.]]
F:\Here\LaneDetection\seg_data\train\0350503e-d3efc96a.png
[[1815755.    4176.]
 [  12553.   10716.]]
F:\Here\LaneDetection\seg_data\train\0553008d-613fe2ed.png
[[2734667.    4213.]
 [  15204.   10716.]]
F:\Here\LaneDetection\seg_data\train\065992a2-8765cbc1.png
[[3629092.    7761.]
 [  28441.   21106.]]
F:\Here\LaneDetection\seg_data\train\081319cb-f5f092a4.png
[[4538825.    8827.]
 [  37367.   22981.]]
F:\Here\LaneDetection\seg_data\train\0968013e-6df14f09.png
[[5435848.   10905.]
 [  47994.   34853.]]
F:\Here\LaneDetection\seg_data\train\123239f3-f0d9bb95.png
[[6322617.   12994.]
 [  69034.   46555.]]
F:\Here\LaneDetection\seg_data\train\1847344d-08a6e901.png
[[7194699.   13055.]
 [ 118205.   46841.]]
F:\Here\LaneDetection\seg_data\train\223046e4-f35461a6.png
[[8096951.   13127.]
 [ 137249.   47073.]]
F:\Here\LaneDetection\seg_data\trai